# Pip Installs and Connect the Drive

In [ ]:
!pip install transformers
!pip install datasets
!pip install pandas

import os
import sys
from google.colab import drive

device='cuda'
drive.mount('/content/drive')
drive_path = 'wav2vec2_final'
final = os.path.join('drive', 'My Drive', drive_path)
print(os.listdir(final))
sys.path.append(final)

file_path = final + '/dataaudio'

# Forming the Dataset

In [ ]:
import numpy
import librosa
import torch
from datasets import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split

paths = []
arr = []
text = []

file_path = final + '/dataaudio'

In [ ]:
for i in range(40):
  paths.append(file_path + f'/{i+1}.wav')

file = file_path + '/text.txt'
with open(file) as f:
    transcription = f.readlines()
f.close()

In [ ]:
for i in range(40):
  transcription[i] = transcription[i].strip('\n')
   #text.append(transcription[i])
  temp,_ = librosa.load(paths[i])
  arr.append(temp)

dataset = {'array': arr, 
           'sampling_rate': 16000, 
           'path': paths, 
           'transcription': transcription
           }

df = pd.DataFrame(dataset)
dataset = Dataset.from_pandas(df)

dataset = dataset.train_test_split(test_size = 0.2, shuffle=False)

# Construct the pre_trained model processor
*Constructs a Wav2Vec2 processor which wraps a Wav2Vec2 feature extractor and a Wav2Vec2 CTC tokenizer into a single processor.*

In [ ]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")

# Forming Data Collator

In [ ]:
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

def prepare_dataset(batch):
    batch["input_values"] = processor(batch["array"], sampling_rate=batch["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["transcription"]).input_ids
    return batch

dataset = dataset.map(prepare_dataset, num_proc=4)

@dataclass
class DataCollatorCTCWithPadding:

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch
        
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

# Constructing and Training the model
*Wav2Vec2ForCTC is used to instantiate a Wav2Vec2 model according to the specified arguments, defining the model architecture. Since the base model is pre-trained on 16 kHz audio, we must make sure our audio sample is also resampled to a 16 kHz sampling rate.*

In [ ]:
from transformers import Wav2Vec2ForCTC, TrainingArguments, Trainer

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base-960h",
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
)

training_args = TrainingArguments(
    output_dir="./results",
    group_by_length=True,
    per_gpu_train_batch_size=8,
    evaluation_strategy="steps",
    num_train_epochs=3,
    fp16=True,
    gradient_checkpointing=True,
    learning_rate=1e-4,
    weight_decay=0.005,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=processor.feature_extractor,
    data_collator=data_collator,
)

torch.cuda.empty_cache()
trainer.train()

# INFERENCE

In [ ]:
audio_input = dataset['test']['array'][5]
input_values = processor(audio_input, sampling_rate = 16000, return_tensors="pt",device='cuda',padding=True).input_values

logits = model(input_values.cuda()).logits
predicted_ids = torch.argmax(logits, dim=-1)

transcription = processor.decode(predicted_ids[0])
print("NEW_DECODED: ", transcription)
print("")


audioText_new = dataset['test']['transcription'][5]


from difflib import SequenceMatcher
m_n = SequenceMatcher(None, audioText_new.upper(), transcription)

print("NEW_ACTUAL: ", audioText_new)
print("") 
print ("NEW_ACCURACY = ", m_n.ratio()*100)
print("")



# Old Data Inference

In [ ]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from transformers import Wav2Vec2ForCTC

processor_old = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model_old = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

audio_input = dataset['test']['array'][5]
input_values = processor_old(audio_input, sampling_rate = 16000, return_tensors="pt",padding=True).input_values

logits = model_old(input_values).logits
predicted_ids = torch.argmax(logits, dim=-1)

transcription = processor_old.decode(predicted_ids[0])

audioText_old = dataset['test']['transcription'][5]


from difflib import SequenceMatcher
m_o = SequenceMatcher(None, audioText_old.upper(), transcription)

print("OLD_DECODED: ", transcription)
print("")
print("OLD_ACTUAL: ", audioText_old)
print("")
print ("OLD_ACCURACY = ", m_o.ratio()*100)




In [ ]:

df = pd.read_csv("/content/drive/MyDrive/wav2vec2_final/dataaudio/speak.csv")
df.loc[df['Sentence'] == audioText_new]

# Graphical Representation

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import gutenberg
import matplotlib.pyplot as plt
file = open("/content/drive/MyDrive/wav2vec2_final/dataaudio/text.txt")
sample = file.read()
m=len(sample)
token = word_tokenize(sample)

In [ ]:
print(token)
t=len(token)
print(t)

In [ ]:
token = word_tokenize(sample)
wlist = []

for i in range(len(token)):
    wlist.append(token[i])

wordfreq = [wlist.count(w) for w in wlist]
print("Pairs\n" + str(zip(token, wordfreq)))

In [ ]:
print(wlist)

In [ ]:
from numpy.core.fromnumeric import argmax
print(token)
print(wordfreq)
len(wordfreq)

  




In [ ]:
wordfreq_ind=list(set(zip(token, wordfreq)))

In [ ]:
wordfreq_ind.sort(key = lambda x: x[1], reverse=True)

In [ ]:
print((wordfreq_ind))

In [ ]:
plt.bar(token, wordfreq)